In [1]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [2]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData

In [3]:
sim = MaaSSim.simulators.simulate(config = 'glance.json')  # run the simulation from a given configuration
sim.runs[0].trips  # access the results
params = MaaSSim.utils.get_config('glance.json')  # load configuration
params.city = "Nootdorp, Netherlands" # modify it
inData = MaaSSim.utils.download_G(inData=inData,params=params)  # download the graph for a new city
sim = MaaSSim.simulators.simulate(inData=inData,params=params) # run the simulation
params.nP = 5 # change number of travellers
inData = MaaSSim.utils.prep_supply_and_demand(inData, params)  # regenerate supply and demand
sim_2 = MaaSSim.simulators.simulate(inData=inData,params=params, logger_level = logging.WARNING) # run the second simulation
print('Total waiting time: {} in first simulation and {} in second.'.format(sim_1.res[0].pax_kpi.loc['sum'].MEETS_DRIVER_AT_PICKUP,
      sim_2.res[0].pax_kpi.loc['sum'].MEETS_DRIVER_AT_PICKUP))  # compare some results
space = DotMap()
space.nP = [5,10,20] # define the search space to explore in experiments
MaaSSim.simulators.simulate_parallel(inData=inData, params=params, search_space = space, logger_level = logging.WARNING) # run parallel experiments
res = MaaSSim.utils.collect_results(params.paths.dumps) # collect results from  parallel experiments

def my_function(**kwargs): # user defined function to represent agent decisions
    veh = kwargs.get('veh', None)  # input from simulation
    sim = veh.sim  # access to simulation object
    if len(sim.runs) > 0:
        if sim.res[last_run].veh_exp.loc[veh.id].nRIDES > 3:
            return False # if I had more than 3 rides yesterday I stay
        else:
            return True # otherwise I leave
    else:
        return True # I do not leave on first day
        
sim = MaaSSim.simulators.simulate(inData=inData,params=params, f_driver_out = my_function, logger_level = logging.INFO) # simulate with my user defined function

14-10-20 15:58:06-WARNING-Setting up 4h simulation at 2020-10-14 15:13:19 for 2 vehicles and 5 passengers in Nootdorp, Netherlands
14-10-20 15:58:06-INFO--------------------	Starting simulation	-------------------
14-10-20 15:58:06-INFO-pax    0  got to dest at                           2020-10-14 15:18:56
14-10-20 15:58:06-INFO-pax    1  got to dest at                           2020-10-14 15:26:01
14-10-20 15:58:06-INFO-pax    2  got to dest at                           2020-10-14 17:45:57
14-10-20 15:58:06-INFO-pax    3  got to dest at                           2020-10-14 17:59:09
14-10-20 15:58:06-INFO-pax    4  got to dest at                           2020-10-14 18:02:49
14-10-20 15:58:06-INFO-veh    2  quitted shift                            2020-10-14 21:13:18
14-10-20 15:58:06-INFO-veh    1  quitted shift                            2020-10-14 21:13:18
14-10-20 15:58:06-INFO--------------------	Simulation over		-------------------
14-10-20 15:58:06-INFO-queue of requests 0
14-10

TypeError: download_G() got an unexpected keyword argument 'params'